<a href="https://githubtocolab.com/gee-community/geemap/blob/master/docs/notebooks/135_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

**Earth Engine Image Segmentation with the Segment Anything Model**

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [9]:
!pip install -U geemap

In [10]:
!pip install segment-geospatial pycrs

  Using cached segment_geospatial-0.12.6-py2.py3-none-any.whl.metadata (11 kB)
  Using cached fiona-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (56 kB)
  Using cached geopandas-1.1.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached ipympl-0.9.7-py3-none-any.whl.metadata (8.7 kB)
  Using cached leafmap-0.48.6-py2.py3-none-any.whl.metadata (16 kB)
  Using cached localtileserver-0.10.6-py3-none-any.whl.metadata (5.2 kB)
  Using cached rasterio-1.4.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.1 kB)
  Using cached rioxarray-0.19.0-py3-none-any.whl.metadata (5.5 kB)
  Using cached sam2-1.1.0-py3-none-any.whl
  Using cached segment_anything_hq-0.3-py3-none-any.whl.metadata (17 kB)
  Using cached segment_anything_py-1.0.1-py3-none-any.whl.metadata (11 kB)
  Using cached timm-1.0.17-py3-none-any.whl.metadata (59 kB)
  Using cached xarray-2025.7.1-py3-none-any.whl.metadata (12 kB)
  Using cached anywidget-0.9.18-py3-none-any.whl.metadat

In [11]:
import ee
import geemap
from samgeo import SamGeo

ModuleNotFoundError: No module named 'samgeo'

In [ ]:
ee.Authenticate()
ee.Initialize(project='tactical-snow-466419-k5')

In [ ]:
Map = geemap.Map()
point = ee.Geometry.Point(-82.3450, 29.6475)
collection = (
    ee.ImageCollection("USDA/NAIP/DOQQ")
    .filterBounds(point)
    .filterDate("2008-01-01", "2018-01-01")
    .filter(ee.Filter.listContains("system:band_names", "N"))
)
image = collection.first()
Map.addLayer(image, {}, "NAIP")
Map.centerObject(point, 16)
Map

In [ ]:
bbox = Map.user_roi_coords()
if bbox is None:
   # bbox = [-122.2666, 37.8682, -122.252, 37.8752]
    bbox = [-82.35, 29.64, -82.33, 29.66]  # Gainesville 附近区域


In [ ]:
geemap.ee_to_geotiff(
    image, "naip.tif", bbox, zoom=17, vis_params={"bands": ["R", "G", "B"]}
)

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    device=None,
    sam_kwargs=None,
)

In [ ]:
sam.generate("naip.tif", output="masks.tif", foreground=True, unique=True)

In [ ]:
sam.show_masks(cmap="binary_r")

In [ ]:
sam.show_anns(axis="off", alpha=1, output="annotations.tif")

In [ ]:
Map.add_raster("annotations.tif", opacity=0.5, layer_name="Masks")
Map

In [ ]:
sam.tiff_to_vector("masks.tif", "masks.shp")

In [ ]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
Map.add_vector("masks.shp", layer_name="Vector", style=style)
Map